In [1]:
from acevedo_clss_and_fcns import * 
model                                 = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
feature_data                          = pd.read_parquet("./results/data/oversampled_augmented_metabolite_data.parquet.gzip")#
feature_names                         = pd.read_csv("./results/data/metabolite_names.csv")
grafo_nx             = cobra_to_networkx(model)


/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
to_remove = [
'nad_c', 'nad_l', 'nad_m', 'nad_r', 'nad_e', 'nad_x', 'nad_n', 'nad_g',
'nadh_c', 'nadh_l', 'nadh_m', 'nadh_r', 'nadh_e', 'nadh_x', 'nadh_n', 'nadh_g',
'nadp_c', 'nadp_l', 'nadp_m', 'nadp_r', 'nadp_e', 'nadp_x', 'nadp_n', 'nadp_g',
'nadph_c', 'nadph_l', 'nadph_m', 'nadph_r', 'nadph_e', 'nadph_x', 'nadph_n', 'nadph_g',

'atp_c', 'atp_l', 'atp_m', 'atp_r', 'atp_e', 'atp_x', 'atp_n', 'atp_g',
'adp_c', 'adp_l', 'adp_m', 'adp_r', 'adp_e', 'adp_x', 'adp_n', 'adp_g',

'gtp_c', 'gtp_l', 'gtp_m', 'gtp_r', 'gtp_e', 'gtp_x', 'gtp_n', 'gtp_g',
'gdp_c', 'gdp_l', 'gdp_m', 'gdp_r', 'gdp_e', 'gdp_x', 'gdp_n', 'gdp_g',

'pi_c', 'pi_l', 'pi_m', 'pi_r', 'pi_e', 'pi_x', 'pi_n', 'pi_g',
'ppi_c', 'ppi_l', 'ppi_m', 'ppi_r', 'ppi_e', 'ppi_x', 'ppi_n', 'ppi_g',
'pppi_c', 'pppi_l', 'pppi_m', 'pppi_r', 'pppi_e', 'pppi_x', 'pppi_n', 'pppi_g',

#'coa_c', 'coa_l', 'coa_m', 'coa_r', 'coa_e', 'coa_x', 'coa_n', 'coa_g',
'co2_c' , 'co2_l' , 'co2_m' , 'co2_r' , 'co2_e' , 'co2_x' , 'co2_n' , 'co2_g' ,

'h2o_c', 'h2o_l', 'h2o_m', 'h2o_r', 'h2o_e', 'h2o_x', 'h2o_n', 'h2o_g',
'h_c', 'h_l', 'h_m', 'h_r', 'h_e', 'h_x', 'h_n', 'h_g',

'fad_c', 'fad_l', 'fad_m', 'fad_r', 'fad_e', 'fad_x', 'fad_n', 'fad_g',
'fadh2_c', 'fadh2_l', 'fadh2_m', 'fadh2_r', 'fadh2_e', 'fadh2_x', 'fadh2_n', 'fadh2_g',

'nh4_c', 'nh4_l', 'nh4_m', 'nh4_r', 'nh4_e', 'nh4_x', 'nh4_n', 'nh4_g',
'so4_c', 'so4_l', 'so4_m', 'so4_r', 'so4_e', 'so4_x', 'so4_n', 'so4_g',
'hco3_c', 'hco3_l', 'hco3_m', 'hco3_r', 'hco3_e', 'hco3_x', 'hco3_n', 'hco3_g',

'o2_c'  , 'o2_l' ,  'o2_m' ,  'o2_r' ,  'o2_e' , 'o2_x' , 'o2_n' , 'o2_g' , 
'h2o2_c', 'h2o2_l', 'h2o2_m', 'h2o2_r', 'h2o2_e', 'h2o2_x', 'h2o2_n', 'h2o2_g',
'o2s_c', 'o2s_l', 'o2s_m', 'o2s_r', 'o2s_e', 'o2s_x', 'o2s_n', 'o2s_g',

'cl_c', 'cl_l', 'cl_m', 'cl_r', 'cl_e', 'cl_x', 'cl_n', 'cl_g',
'k_c', 'k_l', 'k_m', 'k_r', 'k_e', 'k_x', 'k_n', 'k_g',

'na1_c', 'na1_l', 'na1_m', 'na1_r', 'na1_e', 'na1_x', 'na1_n', 'na1_g',
'i_c', 'i_l', 'i_m', 'i_r', 'i_e', 'i_x', 'i_n', 'i_g',
'biomass'
]

## Add metabolite concentrations

In [3]:

grafo_nx_onlyConcen  = add_metabolite_concentration_features(grafo_nx, feature_data, feature_names)


def get_largest_cc(G):
    
  largest_wcc = max(nx.connected_components(nx.Graph(G)), key=len)


  # Create a subgraph SG based on G
  SG = G.__class__()
  SG.add_nodes_from((n, G.nodes[n]) for n in largest_wcc)


  SG.add_edges_from((n, nbr, d)
      for n, nbrs in G.adj.items() if n in largest_wcc
      for nbr, d in nbrs.items() if nbr in largest_wcc)

  SG.graph.update(G.graph)

  assert G.nodes.__len__() >= SG.nodes.__len__()
  assert G.edges.__len__() >= SG.edges.__len__()
  assert SG.nodes.__len__() == len(largest_wcc)
  #assert not SG.is_directed() 
  assert nx.is_connected(nx.Graph(SG))

  return copy.deepcopy(SG)
grafo_nx_onlyConcen.remove_nodes_from(to_remove)

grafo_nx_onlyConcen = get_largest_cc(grafo_nx_onlyConcen)


pyg_graph_onlyConcen = make_PYG_graph_from_grafo_nx(grafo_nx_onlyConcen)

In [4]:
labels                 = torch.tensor(feature_data.label).reshape(len(feature_data.label),1)
pyg_graph_onlyConcen.y = labels
torch.save(pyg_graph_onlyConcen, "./results/graphs_from_PYG_and_NX/PYG_graph_only_Concen.pt")

## Add fluxes

In [5]:

#flux_samples                = pd.read_parquet( "./results/data/flux_samples.parquet.gzip")
flux_samples  = pd.read_parquet("./results/data/flux_samples/flux_samples_v2_10_000.parquet.gzip")


In [6]:


grafo_nx_Concen_plus_Fluxes = add_flux_features(grafo_nx_onlyConcen, flux_samples, feature_data)

In [7]:
pyg_graph_Concen_plus_Fluxes = make_PYG_graph_from_grafo_nx(grafo_nx_Concen_plus_Fluxes)

In [8]:
pyg_graph_Concen_plus_Fluxes.y = labels
torch.save(pyg_graph_Concen_plus_Fluxes, "./results/graphs_from_PYG_and_NX/PYG_graph_Concen_plus_Fluxes.pt")

In [9]:
list(grafo_nx_Concen_plus_Fluxes.nodes).index('phe_L_c')


nx.write_gpickle(grafo_nx_Concen_plus_Fluxes, "./results/graphs_from_PYG_and_NX/grafo_nx_Concen_plus_Fluxes.gpickle")